## Minimal SmartNoiseSQL example on the income dataset

(Just to check if it's working as intended)

In [2]:
from fso_sdd_demo.client import Client

APP_URL = "http://sdd_server_dev:80" # Onyxia: "https://sdd-server-demo.lab.sspcloud.fr"
USER_NAME = "Dr. FSO"
DATASET_NAME = "FSO_INCOME_SYNTHETIC"
client = Client(url=APP_URL, user_name = USER_NAME, dataset_name = DATASET_NAME)

In [3]:
income_metadata = client.get_dataset_metadata()['']['Schema']['Table']
income_metadata

{'max_ids': 1,
 'region': {'type': 'int'},
 'eco_branch': {'type': 'int'},
 'profession': {'type': 'int'},
 'education': {'type': 'int'},
 'age': {'type': 'int'},
 'sex': {'type': 'int'},
 'income': {'type': 'float', 'lower': 1000, 'upper': 100000}}

In [4]:
NB_ROWS = 200
SEED = 0

df_dummy = client.get_dummy_dataset(
    nb_rows = NB_ROWS, 
    seed = SEED
)

print(df_dummy.shape)
df_dummy.head()

(200, 7)


,region,eco_branch,profession,education,age,sex,income
0,-7268,-8759,4470,-7097,2279,4470,99981.049234
1,799,903,-4654,-6748,-2700,1345,20528.687956
2,-155,-5597,-5224,-3552,9480,8039,53955.911640
3,9648,7582,3729,4274,-178,-7578,29734.556213
4,3123,9381,-5878,-2574,9842,8165,31113.182182


In [5]:
import opendp_polars.prelude as dp
import opendp_polars.transformations as trans
import opendp_polars.measurements as meas

dp.enable_features("contrib")

columns = ["region", "eco_branch", "profession", "education", "age", "sex", "income"]

income_min = income_metadata["income"]["lower"]
income_max = income_metadata["income"]["upper"]

num_rows_pipeline = (
    trans.make_split_dataframe(separator=",", col_names=columns) >>
    trans.make_select_column(key="income", TOA=str) >>
    trans.then_count() >>
    meas.then_laplace(scale=0.5) # scale arbitrary
)

num_rows = client.opendp_query(
    opendp_pipeline = num_rows_pipeline
)

num_rows

{'requested_by': 'Dr. FSO',
 'state': 'Query successful.',
 'query_response': 2032544,
 'spent_epsilon': 2.0,
 'spent_delta': 0,
 'remaining_epsilon': 41.0,
 'remaining_delta': 0.005}

This here does not work, feel free to comment/update, etc..

In [10]:
income_average_pipeline = (
    trans.make_split_dataframe(separator=",", col_names=columns) >>
    trans.make_select_column(key="income", TOA=str) >>
    trans.then_cast_default(TOA=float) >>
    trans.then_clamp(bounds=(income_min, income_max)) >>
    trans.then_bounded_resize(size=num_rows, bounds=(income_min, income_max), constant=0) >> # how do you choose constant?
    trans.then_sized_bounded_mean(size=num_rows, bounds=(income_min, income_max)) >>
    meas.then_laplace(size=0.5)
)

income_average = client.opendp_query(
    opendp_pipeline = income_average_pipeline
)

TypeError: then_resize() got an unexpected keyword argument 'bounds'